# Text Preprocessing

Download NLTK libraries

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

Punction, sentence tokenizer, pos tagging, lemmatization process

In [5]:
import nltk
import string
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

def punctionProcess(text):
    out = str(text).maketrans('','', string.punctuation)
    result = str(text).translate(out)
    
    return result

#punktTokenizer for sentence detection
def punktSentenceTokenizer(paragraph):
    sent_tokenizer = nltk.tokenize.PunktSentenceTokenizer()
    sentences = sent_tokenizer.tokenize(paragraph)
    updateSentence = solveSentenceTokenizer(sentences)
    if len(updateSentence) == 0:
        updateSentence = sentences

    return updateSentence


#POS Tagging
def posTag(text):
    txt = word_tokenize(text)
    pos = nltk.pos_tag(txt)

    return pos

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
# Lemmatize with POS Tag
from nltk.corpus import wordnet
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


#Lemmatization process
lemmatizer=WordNetLemmatizer()
def lemmaProcess(sentence):       
    input_str=word_tokenize(sentence)
    txt = ''
    for word in input_str:
        txt += lemmatizer.lemmatize(word, get_wordnet_pos(word)) + ' ' #, get_wordnet_pos(word)
    sentence = str(txt).strip()

    return sentence


#developed solve for sentence tokenizer error
def solveSentenceTokenizer(sentences):
    firstStage = []
    ind = 0
    indList = []
    last = []
    #If next sentence first word's first letter is lower or numeric char, combine with sentences
    s = ''
    for sentence in sentences:
        if ind+1 < len(sentences):
            words = word_tokenize(sentences[ind+1])
            word = words[0]
            if str(word).islower() == True or str(word).isnumeric() == True:
                firstStage.append(sentence+ ' ' + sentences[ind+1])
                s += sentence+ ' ' + sentences[ind+1]
                last.append(ind+1)
                if ind+1 - last[len(last)-2] == 1:
                    sent = firstStage[len(firstStage)-2]
                    firstStage.remove(firstStage[len(firstStage)-2])
                    firstStage.remove(firstStage[len(firstStage)-1])
                    sent += ' ' + sentences[ind+1]
                    firstStage.append(sent)
                    s += sent
            else:
                firstStage.append(sentence)  
        else:
                firstStage.append(sentence) 
        ind += 1
    for ind in range(0, len(firstStage)-1):
        if str(firstStage[ind]).find(firstStage[ind+1]) != -1:
            indList.append(ind+1)
    indList.sort(reverse=True)
    for ind in indList:
        firstStage.remove(firstStage[ind])
    #If sentence containing max 2 word, combines with next sentences
    secondStage = []
    ind = 0
    for sentence in firstStage:
        words = word_tokenize(sentence) 
        if len(words) < 3:
            if  ind+1 < len(firstStage):
                secondStage.append(sentence + ' ' + firstStage[ind+1])
        else:
            if len(secondStage) > 0:
                if str(secondStage[len(secondStage)-1]).find(sentence) == -1:
                    secondStage.append(sentence)
            else:
                secondStage.append(sentence)
        ind += 1
    #Previous word is uppercase, but last word's first character is uppercase with max 3 length => combine with next sentences 
    lastSentences = []
    ind = 0
    for update in secondStage:
        words = word_tokenize(update) 
        if len(words) >= 2:
            word = words[len(words)-2]
            if len(lastSentences) != 0:
                if str(lastSentences[len(lastSentences)-1]).find(update) != -1:
                    ind += 1
                    continue
            if str(word[0]).isupper() == True and len(word) < 4 and len(words) >= 3:
                word = words[len(words)-3]
                if str(word[0]).islower() == True and len(secondStage) > ind+1:
                    lastSentences.append(update + ' ' + secondStage[ind+1])
                else:
                    lastSentences.append(update)
            else:
                lastSentences.append(update)
        ind += 1
    return lastSentences

Named Entity Recognition process

In [2]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm

nlp = en_core_web_sm.load()
#NER process for preprocessing
def NerRecognition(article, answ_list):
    article = nlp(article)
    
    items = [x.text for x in article.ents]   
    ner_list = Counter(items).keys()
    sw = stopwords.words('english') 
    for ner in ner_list:
        if str(ner).find('\'s') != -1:
            indx = str(ner).index('\'s')
            ner = ner[:indx]
        if str(ner).find(' ') != -1:                  
            ner_ = str(ner).split(' ')
            islow = False
            if str(ner_[len(ner_)-1]).islower() == True:
                islow = True
            try:
                ind = sw.index(ner_[0])
            except:
                uri = ""
                for i in ner_:
                    if islow == False:
                        try:
                            try:
                                ner_list.index(i)
                            except:
                                ind = answ_list.index(i)
                                answ_list.remove(str(i).strip())
                            uri += str(i)  + "_" 
                        except:
                            uri += str(i) + "_"                            
                if uri != "": answ_list.append(str(uri[:-1]))
        else:
            for ans in answ_list:
                if ans.find(ner) != -1:
                    answ_list.remove(ans)
                    answ_list.append(ner)
                    break
    return answ_list

# Demo Processes

In [3]:
from collections import OrderedDict
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from spacy import displacy
from collections import Counter

Answer type detection function

In [4]:
def findAnswerType(pos_txt):
	ansType = ''
	if pos_txt == 'people' or pos_txt == 'person':
		ansType = 'PERSON'
	if pos_txt == 'nationality' or pos_txt == 'religion' or pos_txt == 'politic':
		ansType = 'NORP'
	if pos_txt == 'country' or pos_txt == 'city' or pos_txt == 'state':
		ansType = 'GPE'
	if pos_txt == 'company' or pos_txt == 'agency' or pos_txt == 'institution':
		ansType = 'ORG'
	if pos_txt == 'building' or pos_txt == 'airport' or pos_txt == 'highway' or pos_txt == ' bridge':
		ansType = 'FAC'
	if pos_txt == 'location' or pos_txt == 'range':
		ansType = 'LOC'
	if pos_txt == 'hurricane' or pos_txt == 'battle' or pos_txt == 'war':
		ansType = 'EVENT'
	if pos_txt == 'song' or pos_txt == 'title':
		ansType = 'WORK_OF_ART'
	if pos_txt == 'language':
		ansType = 'LANGUAGE'
	if pos_txt == 'time' or pos_txt == 'date' or pos_txt == 'period' or pos_txt == 'duration' or pos_txt == 'year' or pos_txt == 'month' or pos_txt == 'day':
		ansType = 'DATE'
	if pos_txt == 'percentage':
		ansType = 'PERCENT'
	if pos_txt == 'measurement' or pos_txt == 'weight' or pos_txt == 'distance':
		ansType = 'QUANTITY'
		
	return ansType

Define RNP methods

In [6]:
def demo(related_sentence, answerTerm, questionSearch, quesNormal, totalFindAnswer, methodNo, isLemmatization):
    processes = ''
    sentence = related_sentence
    try:         
        question = questionSearch.split(',')
        if isLemmatization == True:          
            sentence = lemmaProcess(sentence)
            quesNormal = lemmaProcess(quesNormal)
            answerTerm = lemmaProcess(answerTerm)
        
        sentenceTerm = []
        #Remove&Compare method
        if methodNo == 1:           
            question = punctionProcess(quesNormal)  
            question = question.lower().strip().split(' ')
            sentence = punctionProcess(sentence)  
            sentence = sentence.lower().strip().split(' ')
            answer = punctionProcess(answerTerm)   
            answer = answer.lower().strip().split(' ')
            sw = stopwords.words('english')
            processes += '\nRemoving question words in sentence: ' 
            for ques in question:                
                try: 
                    sentence.remove(ques)
                    processes += ques + ','
                except: 
                    for synset in wordnet.synsets(ques):
                        cnt = False
                        for syn in synset.lemmas():
                            if str(related_sentence).find(syn.name()) != -1:
                                try:
                                    sentence.remove(str(syn.name()))
                                    cnt = True
                                    break
                                except: continue
                        if cnt == True:
                            break
                    continue
            processes += '\n\nRemoving stopwords in sentence: '
            for stop in sw:
                try: 
                    sentence.remove(stop)
                    processes += stop + ','                    
                except: 
                    continue
            processes += '\n\nRemoving stopwords in answer: '
            for stop in sw:
                try: 
                    answer.remove(stop)
                    processes += stop + ',' 
                except: 
                    continue
            if len(sentence) == len(answer):
                sent = ''
                ans = ''
                for s in sentence:
                    sent += s + ' '
                for a in answer:
                    ans += a + ' '
                if sent.strip() == ans.strip():
                    totalFindAnswer += 1

        #Searching with NER method
        if methodNo == 2:
            text = str(quesNormal).lower()
            ansType = ''
            if text.find('who') != -1:
                ansType = 'PERSON'
            if text.find('when') != -1:
                ansType = 'DATE'
            if text.find('how many') != -1:
                ansType = 'QUANTITY'
            if text.find('how much') != -1:
                ansType = 'QUANTITY, MONEY'
            if text.find('where') != -1:
                ansType = 'GPE'
            pos = posTag(quesNormal.lower())
            if len(pos)> 2:
                for i in range(len(pos)-1):
                    if pos[i][0] == 'what' or pos[i][0] == 'which' or pos[i][0] == 'where' or pos[i][0] == 'whose':
                        ansType = findAnswerType(pos[i+1][0])
                if ansType == '':
                    for i in range(len(pos)-1):
                        ansType = findAnswerType(pos[i+1][0])
                if ansType != '':                   
                    sentence = nlp(sentence)
                    labels = [x.label_ for x in sentence.ents]
                    sentences = [x for x in sentence.sents]
                    processes += 'Question\'s NER label: ' + ansType + '\n\n'
                    processes += 'Related sentence\'s NER labels:\n' 
                    answer = ''
                    ind = 0
                    for k in nlp(str(sentences[0])).ents:
                        processes += str(k) + ': ' + str(labels[ind]) + '\n' 
                        ind += 1
                    ind = 0
                    for k in nlp(str(sentences[0])).ents:
                        if ansType.find(labels[ind]) != -1 and text.find(str(k)) == -1:
                            answer = str(k)
                            break
                        ind += 1
                    if answer == answerTerm or answer == answerTerm.lower():
                        totalFindAnswer += 1

        #Searching with POS method
        if methodNo == 3:
            pos = posTag(quesNormal.lower())
            ansType = ''
            if len(pos) > 2:
                for i in range(len(pos)-1):         
                    if pos[i][1] == 'NNP' or pos[i][1] == 'WP$':
                        if pos[i][0] == 'whose': ansType = "NNP"
                    if pos[i][1] == 'WP' or pos[i][1] == 'WP':
                        if pos[i][0] == 'who': ansType = "NNP"
                        if pos[i][0] == 'what': ansType = "NN"
                    if pos[i][1] == "WRB":
                        if pos[i][0] == 'when': ansType = "CD"
                        if pos[i][0] == 'how': 
                            if pos[i+1][1] == 'JJ': ansType = "CD"
                            else: ansType = "JJ,RB"
                        if pos[i][0] == 'where': ansType = "NN"
                        if pos[i][0] == 'why': ansType = "IN"
                    if pos[i][1] == "WDT":
                        ansType = "NN"
                    if ansType != '':
                        break
        
                if ansType != '':
                    processes += 'Question\'s POS tag: ' + ansType + '\n\n'
                    processes += 'Removing question terms in sentence: '
                    question = quesNormal[:-1].split(' ')
                    for ques in question:
                        if sentence.find(ques) != -1:
                            ind = sentence.index(ques)
                            processes += ques + ','
                            sentence = sentence[:ind-1] + sentence[ind+len(ques):]                 
                    posSent = posTag(sentence.strip())
                    processes += '\nRelated sentence\'s POS tags:\n'
                    ans = ''
                    ind = 0
                    for key, p in posSent:
                        processes += str(key) + ': ' + str(p) + '\n' 
                        ind += 1
                    ind = 0
                    for key, p in posSent:
                        if ansType.find(p) != -1:
                            ans += str(key) + ' '
                            if ind+1 < len(posSent)-1:
                                for k in range(ind+1,len(posSent)):
                                    if ansType.find(str(posSent[k][1])) != -1:
                                        ans += str(posSent[k][0]) + ' '
                                    else: break
                            break
                        ind += 1
                    if ans != '':
                        if ans[:-1] == answerTerm or ans[:-1] == answerTerm.lower():
                            totalFindAnswer += 1        

    except Exception as e:
        print('Error Answer detection: ' + str(e) + ' - method:' + str(methodNo))        
    return totalFindAnswer, processes

In [7]:
def defineQuestionTerm(preProcess):
    questionSearch = ''
    for terms in preProcess:
        for term in terms:
            if str(term).find('_') != -1:
                term = str(term).replace('_',' ')
            questionSearch += term + ','
    questionSearch = questionSearch[:-1]
    return questionSearch

Search question words in sentences with question term percentage(QTP)

In [10]:
def searchWord(text,paragraph,lemma):
    texts = []
    textSearch = ''
    path = ''
    try:
        if str(text).find(',') != -1:
            texts = str(text).split(',')
        else:
            texts.append(text)  
        par = paragraph          
        sentences = punktSentenceTokenizer(par)
        s_ind = 0
        rateList = []
        textList = []
        for sentence in sentences:         
            sent = punctionProcess(str(sentence))
            if lemma == True:
                sent = lemmaProcess(sent)
                sentence = sent
            c = 0
            qaText = [] 
            qTerm = []
            for i in range(0,len(texts)):
                texts[i] = str(texts[i]).strip()
                if str(sent.lower()).find(texts[i].lower()) != -1:
                    c += 1
                    qTerm.append(str(texts[i]))

            if c != 0:
                textSearch += 'Find question:' + str(qTerm) + ', QCount:' + str(c) + '\nQuestion rate:%' + str(c/len(texts)*100) + '\n' + str(s_ind+1) + '. sentence - ' + sentence + '\n\n'
                rateList.append(c/len(texts)*100)
                textList.append('Find question:' + str(qTerm) + ', QCount:' + str(c) + '\nQuestion rate:%' + str(c/len(texts)*100) + '\n' +  str(s_ind+1) + '. sentence - ' + sentence + '\n\n')
            s_ind += 1
        if len(rateList) != 0:
            path += findPath(rateList,textList)
    except Exception as e:
        print('SearchWord:' + str(e) + ', text:' + str(texts))
    return textSearch, path

Sorted all sentence according to QTP value

In [9]:
def findPath(rateList,textList):
    sortList = []
    sortTxt = ''
    for rate in rateList:
        value = max(rateList)
        ind = rateList.index(value)
        sortList.append(ind)
        rateList[ind] = 0
    for sort in sortList:
        sortTxt += textList[sort] + '\n'
    return sortTxt

Selected candidate sentence

In [40]:
def demoProcess(question, paragraph, answer, lemma):
  question = punctionProcess(question)
  X_list = word_tokenize(question) 
  preProcess = [X_list]
   
  questionSearch = defineQuestionTerm(preProcess)
  processes = 'Question terms: ' + questionSearch + '\n'
  textSearch, path = searchWord(questionSearch, paragraph, lemma)
  texts = path.split('\n')
  print('Sorted sentence: ' + str(texts))
  txtInd = 0
  sel_sentence = ''
  for txt in texts:
      if (str(txt.lower()).find(answer.lower()) != -1 or str(txt).find(answer) != -1):
          sentence = str(txt).split('sentence - ')
          sentence = str(sentence[1]).strip()
          sel_sentence = str(sentence).replace('\n', '').strip()
          break

  return questionSearch, sel_sentence

#Running Examples

In [12]:
paragraph_examples = 'Tarkan Tevetoğlu is a Turkish singer and songwriter. Tarkan was born on 17 October 1972 in Alzey, West Germany, to Neşe and Ali Tevetoğlu. The name Tarkan is said to originate from an ancient Turkic King. Tarkan\'s first album, Yine Sensiz, was released on cassettes on 26 December 1992. Since the early 1990s, with the successful sales of his albums, he has been a prominent figure of pop music, recognized in Turkey and worldwide.'
question_examples = ['Who is Tarkan Tevetoğlu?','When was Tarkan born?','Whose name does Tarkan originate from?']
answer_examples = ['Turkish singer and songwriter', '17 October 1972', 'Turkic King']
  
lemma = False
question = ''
preProcess = '' 
answer = ''
paragraph = paragraph_examples

Remove & Compare example

In [41]:
question = question_examples[0]
answer = answer_examples[0]
response = answer_examples[0]
answ = answer_examples[0].lower().strip().split(' ')
sw = stopwords.words('english')
for stop in sw:
    try: 
        answ.remove(stop)                 
    except: continue
res = ''
for ans in answ:
    res += ans + ' '
answer = res

questionSearch, sel_sentence = demoProcess(question, paragraph, response, lemma)
print('Candidate sentence: ' + sel_sentence)
totalFindAnswer = 0
findCount = totalFindAnswer
totalFindAnswer, process = demo(sel_sentence, answer, questionSearch, question, totalFindAnswer, 1, lemma)
if totalFindAnswer != findCount:
  print('RC: Answer found successfully, answer: ' + answer)

Sorted sentence: ["Find question:['is', 'Tarkan', 'Tevetoğlu'], QCount:3", 'Question rate:%75.0', '1. sentence - Tarkan Tevetoğlu is a Turkish singer and songwriter.', '', '', "Find question:['Tarkan', 'Tevetoğlu'], QCount:2", 'Question rate:%50.0', '2. sentence - Tarkan was born on 17 October 1972 in Alzey, West Germany, to Neşe and Ali Tevetoğlu.', '', '', "Find question:['is', 'Tarkan'], QCount:2", 'Question rate:%50.0', '3. sentence - The name Tarkan is said to originate from an ancient Turkic King.', '', '', "Find question:['Tarkan'], QCount:1", 'Question rate:%25.0', "4. sentence - Tarkan's first album, Yine Sensiz, was released on cassettes on 26 December 1992.", '', '', "Find question:['is'], QCount:1", 'Question rate:%25.0', '5. sentence - Since the early 1990s, with the successful sales of his albums, he has been a prominent figure of pop music, recognized in Turkey and worldwide.', '', '', '']
Candidate sentence: Tarkan Tevetoğlu is a Turkish singer and songwriter.
RC: Answe

SNER example

In [47]:
question = question_examples[1]
answer = answer_examples[1]

questionSearch, sel_sentence = demoProcess(question, paragraph, answer, lemma)
totalFindAnswer = 0
findCount = totalFindAnswer
totalFindAnswer, process = demo(sel_sentence, answer, questionSearch, question, totalFindAnswer, 2, lemma)
if totalFindAnswer != findCount:
  print(process)
  print('SNER: Answer found successfully, answer: ' + answer)

Sorted sentence: ["Find question:['was', 'Tarkan', 'born'], QCount:3", 'Question rate:%75.0', '2. sentence - Tarkan was born on 17 October 1972 in Alzey, West Germany, to Neşe and Ali Tevetoğlu.', '', '', "Find question:['was', 'Tarkan'], QCount:2", 'Question rate:%50.0', "4. sentence - Tarkan's first album, Yine Sensiz, was released on cassettes on 26 December 1992.", '', '', "Find question:['Tarkan'], QCount:1", 'Question rate:%25.0', '1. sentence - Tarkan Tevetoğlu is a Turkish singer and songwriter.', '', '', "Find question:['Tarkan'], QCount:1", 'Question rate:%25.0', '3. sentence - The name Tarkan is said to originate from an ancient Turkic King.', '', '', '']
Question's NER label: DATE

Related sentence's NER labels:
17 October 1972: DATE
Alzey: ORG
West Germany: GPE
Ali Tevetoğlu: PERSON

SNER: Answer found successfully, answer: 17 October 1972


SPOS example

In [49]:
question = question_examples[2]
answer = answer_examples[2]

questionSearch, sel_sentence = demoProcess(question, paragraph, answer, lemma)
totalFindAnswer = 0
findCount = totalFindAnswer
totalFindAnswer, process = demo(sel_sentence, answer, questionSearch, question, totalFindAnswer, 3, lemma)
if totalFindAnswer != findCount:
  print(process)
  print('SPOS: Answer found successfully, answer:' + answer)

Sorted sentence: ["Find question:['name', 'Tarkan', 'originate', 'from'], QCount:4", 'Question rate:%66.66666666666666', '3. sentence - The name Tarkan is said to originate from an ancient Turkic King.', '', '', "Find question:['Tarkan'], QCount:1", 'Question rate:%16.666666666666664', '1. sentence - Tarkan Tevetoğlu is a Turkish singer and songwriter.', '', '', "Find question:['Tarkan'], QCount:1", 'Question rate:%16.666666666666664', '2. sentence - Tarkan was born on 17 October 1972 in Alzey, West Germany, to Neşe and Ali Tevetoğlu.', '', '', "Find question:['Tarkan'], QCount:1", 'Question rate:%16.666666666666664', "4. sentence - Tarkan's first album, Yine Sensiz, was released on cassettes on 26 December 1992.", '', '', '']
Question's POS tag: NNP

Removing question terms in sentence: name,Tarkan,originate,from,
Related sentence's POS tags:
The: DT
is: VBZ
said: VBD
to: TO
an: DT
ancient: JJ
Turkic: NNP
King: NNP
.: .

SPOS: Answer found successfully, answer:Turkic King
